# PySpark - Apply Function


In [1]:
%%html
<style>
table {float:left}
</style>

In [2]:
%%html
<style>
div.output_area pre {
    white-space: pre;
}
</style>

In [3]:
import os
import sys
import gc
import numpy as np

# Constant

In [4]:
USER = !whoami
USER = USER[0]

PYSPARK_PYTHON_PATH = f"/home/{USER}/venv/ml/bin/python3"
HADOOP_HOME = "/opt/hadoop/hadoop-3.2.2"
SPARK_HOME = "/opt/spark/spark-3.1.2"

#  Environemnt Variables

## Hadoop

In [5]:
os.environ['HADOOP_CONF_DIR'] = f"${HADOOP_HOME}/etc/hadoop"

In [6]:
%%bash -s "$HADOOP_HOME"
export HADOOP_CONF_DIR="$1/etc/hadoop"
ls $HADOOP_CONF_DIR | head -n 3

capacity-scheduler.xml
configuration.xsl
container-executor.cfg


## PYTHONPATH

Refer to the **pyspark** modules to load from the ```$SPARK_HOME/python/lib``` in the Spark installation.

* [PySpark Getting Started](https://spark.apache.org/docs/latest/api/python/getting_started/install.html)

> Ensure the SPARK_HOME environment variable points to the directory where the tar file has been extracted. Update PYTHONPATH environment variable such that it can find the PySpark and Py4J under SPARK_HOME/python/lib. One example of doing this is shown below:

```
export PYTHONPATH=$(ZIPS=("$SPARK_HOME"/python/lib/*.zip); IFS=:; echo "${ZIPS[*]}"):$PYTHONPATH
```

Alternatively install **pyspark** with pip or conda locally which installs the Spark runtime libararies (for standalone).

* [Can PySpark work without Spark?](https://stackoverflow.com/questions/51728177/can-pyspark-work-without-spark)

> As of v2.2, executing pip install pyspark will install Spark. If you're going to use Pyspark it's clearly the simplest way to get started. On my system Spark is installed inside my virtual environment (miniconda) at lib/python3.6/site-packages/pyspark/jars  
> PySpark has a Spark installation installed. If installed through pip3, you can find it with pip3 show pyspark. Ex. for me it is at ~/.local/lib/python3.8/site-packages/pyspark. This is a standalone configuration so it can't be used for managing clusters like a full Spark installation.

In [7]:
# os.environ['PYTHONPATH'] = "/opt/spark/spark-3.1.2/python/lib/py4j-0.10.9-src.zip:/opt/spark/spark-3.1.2/python/lib/pyspark.zip"
sys.path.extend([
    f"{SPARK_HOME}/python/lib/py4j-0.10.9-src.zip",
    f"{SPARK_HOME}/python/lib/pyspark.zip"
])

## PYSPARK_PYTHON

In [8]:
os.environ['PYSPARK_PYTHON'] = PYSPARK_PYTHON_PATH

## PySpark packages

Execute after the PYTHONPATH setup.

In [9]:
import pyspark.sql 
from pyspark.sql.types import *
from pyspark.sql.functions import (
    col,
    lit,
    when,
    isnan,
    isnull,
    lower,
    upper,
    concat,
    udf,
    array,
    avg,
    stddev,
    to_date,
    date_format,
    months_between,
    add_months,
)

---
# Spark Session


In [10]:
from pyspark.sql import SparkSession

In [11]:
# For YARN cluster
#    .config('spark.yarn.appMasterEnv.PYSPARK_PYTHON', f"/home/{USER}/venv/ml/bin/python3")\

spark = SparkSession.builder\
    .master('yarn') \
    .config('spark.submit.deployMode', 'client') \
    .config('spark.debug.maxToStringFields', 100) \
    .config('spark.executor.memory', '2g') \
    .config('spark.yarn.executorEnv.PYSPARK_PYTHON', f"/home/{USER}/venv/ml/bin/python3")\
    .getOrCreate()

2022-02-24 06:28:14,284 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-02-24 06:28:17,077 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [12]:
NUM_CORES = 4
NUM_PARTITIONS = 3

spark.conf.set("spark.sql.shuffle.partitions", NUM_CORES * NUM_PARTITIONS)
spark.conf.set("spark.default.parallelism", NUM_CORES * NUM_PARTITIONS)
spark.conf.set('spark.sql.legacy.timeParserPolicy', 'LEGACY')

---

---
# Stop Spark Session

In [13]:
spark.stop()



# Cleanup

In [14]:
del spark
gc.collect()

105